In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json

import common
from search_tools import create_search_tools

evidently_search_tools = create_search_tools()

In [7]:
from google import genai
from google.genai import types

gemini_client = genai.Client()

In [8]:
import inspect

search_tool = {
    "name": evidently_search_tools.search.__name__,
    "description": inspect.getdoc(evidently_search_tools.search),
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "query parameter"
            }
        },
        "required": [
            "query"
        ]
    }
}

get_file_tool = {
    "name": evidently_search_tools.get_file.__name__,
    "description": inspect.getdoc(evidently_search_tools.get_file),
    "parameters": {
        "type": "object",
        "properties": {
            "filename": {
                "type": "string",
                "description": "filename parameter"
            }
        },
        "required": [
            "filename"
        ]
    }
}

In [9]:
tool_pairs = [
    (evidently_search_tools.search, search_tool),
    (evidently_search_tools.get_file, get_file_tool)
]

In [10]:
tools = [t for (_, t) in tool_pairs]
tools_index = {t["name"]: f for (f, t) in tool_pairs}

In [28]:
def make_call(tool_call):
    name = tool_call.name

    if name in tools_index:
        func = tools_index[name]
        arguments = tool_call.args
        result = func(**arguments)
        if not isinstance(result, dict):
            result = {"result": result}
    else:
        result = {
            "error": f"Tool '{name}' not found in tools index"
        }

    return types.Part(
        function_response=types.FunctionResponse(
            name=name,
            response=result,
            id=tool_call.id,
        )
    )

In [29]:
from dataclasses import dataclass

@dataclass
class ToolCall:
    name: str
    args: str
    id: str

tool_call = ToolCall(name="search", args={"query": "dashboard"}, id="1234")
tool_call_result = make_call(tool_call)
tool_call_result

Part(
  function_response=FunctionResponse(
    id='1234',
    name='search',
    response={
      'result': [
        {
          'content': {
            'matches': [<... 3 items at Max depth ...>],
            'total_matches': 11
          },
          'description': 'How to design your Dashboard with custom Panels.',
          'filename': 'docs/platform/dashboard_add_panels_ui.mdx',
          'title': 'Add dashboard panels (UI)'
        },
        {
          'content': {
            'matches': [<... 3 items at Max depth ...>],
            'total_matches': 27
          },
          'description': 'How to design your Dashboard with custom Panels.',
          'filename': 'docs/platform/dashboard_add_panels.mdx',
          'title': 'Add dashboard panels (API)'
        },
        {
          'content': {
            'matches': [<... 3 items at Max depth ...>],
            'total_matches': 62
          },
          'description': 'Overview of the available monitoring Panels.',
         

In [13]:
messages = [
    types.Content(
        parts=[types.Part(text=common.DEFAULT_QUESTION)],
        role="user"
    )
]

# 1st call - initial

response = gemini_client.models.generate_content(
    model='models/gemini-3-flash-preview',
    contents=messages,
    config=types.GenerateContentConfig(
        tools=[types.Tool(function_declarations=tools)],
        system_instruction=common.SINGLE_TOOL_SYSTEM_PROMPT,
    )
)

In [20]:
parts = response.candidates[0].content.parts
parts

[Part(
   function_call=FunctionCall(
     args={
       'query': 'create dashboard'
     },
     name='search'
   ),
   thought_signature=b'\x12\x90\x04\n\x8d\x04\x01\xbe>\xf6\xfb1\xfa1\xc0\xe4\xac>\xf2\xd7\xb3\x8c5\xf6N\x95\x1bJ\xff\x01\xd2e\xcf\x13\xd1-\xaeNqVn\xc8\xcf]U\xb9\xb4\x9f\xd3\x1c\xcb6\n\xbb\x0e\x8b\xba\xde\x83\xe201cq\xde\x96\x82\xf9i\xf7\x1f\xa9v\xcf\xc8E\x03\x07\xb4\x9f\xadO\x0bXV(\xbeh\x88\xe4\xe8\x8d\x0bv\xd5\x1a*...'
 )]

In [22]:
tool_call = [p.function_call for p in parts if p.function_call is not None][0]
tool_call

FunctionCall(
  args={
    'query': 'create dashboard'
  },
  name='search'
)

In [23]:
print(f"Tool call: {tool_call.name}({tool_call.args})")

Tool call: search({'query': 'create dashboard'})


In [30]:
tool_call_result = make_call(tool_call)
tool_call_result

Part(
  function_response=FunctionResponse(
    id='1234',
    name='search',
    response={
      'result': [
        {
          'content': {
            'matches': [<... 3 items at Max depth ...>],
            'total_matches': 11
          },
          'description': 'How to design your Dashboard with custom Panels.',
          'filename': 'docs/platform/dashboard_add_panels_ui.mdx',
          'title': 'Add dashboard panels (UI)'
        },
        {
          'content': {
            'matches': [<... 3 items at Max depth ...>],
            'total_matches': 27
          },
          'description': 'How to design your Dashboard with custom Panels.',
          'filename': 'docs/platform/dashboard_add_panels.mdx',
          'title': 'Add dashboard panels (API)'
        },
        {
          'content': {
            'matches': [<... 3 items at Max depth ...>],
            'total_matches': 62
          },
          'description': 'Overview of the available monitoring Panels.',
         

In [31]:
messages.append(types.Content(role="model", parts=parts))
messages.append(types.Content(role="user", parts=[tool_call_result]))

In [32]:
# 2nd call

response = gemini_client.models.generate_content(
    model='models/gemini-3-flash-preview',
    contents=messages,
    config=types.GenerateContentConfig(
        tools=[types.Tool(function_declarations=tools)],
        system_instruction=common.SINGLE_TOOL_SYSTEM_PROMPT,
    )
)

In [33]:
parts = response.candidates[0].content.parts
parts

[Part(
   function_call=FunctionCall(
     args={
       'filename': 'docs/platform/dashboard_overview.mdx'
     },
     name='get_file'
   ),
   thought_signature=b'\x12\xf9\x05\n\xf6\x05\x01\xbe>\xf6\xfb5\x08\x15\r?\xb9\x1aC\xff\xd6\x9c\xbeM\x16\x80\xd5\x7fd\xe0\xb9#\x87p\xb5\xad\x16\xa9;\xcf\xc1\xee\x93?I\x05\x9b\xde\xda\x1f\x97WI\xe1\xdbO\x90\x02#H\xbb,\xa2\x1c\xee\xf6\xb7\xaf\x8amtw\xb3\xcd\x7f\xb0T\x9a\xe7\x1d+w\xee\xbd\x89BT\xaf\x8f\x7f\x19_o%\xfa\x82\xbf...'
 )]

In [34]:
tool_call_results = []

for part in parts:
    if part.function_call is not None:
        function_call = part.function_call
        print(f'executing {function_call.name}({function_call.args})...')
        tool_call_output = make_call(function_call)
        tool_call_results.append(tool_call_output)

    if part.text is not None:
        text = message.text
        print('ASSISTANT:', text)

executing get_file({'filename': 'docs/platform/dashboard_overview.mdx'})...


In [35]:
messages.append(types.Content(role="model", parts=parts))
messages.append(types.Content(role="user", parts=tool_call_results))

In [38]:
messages.append(types.Content(
    role="user",
    parts=[types.Part(text="SYSTEM: produce the final output now!")]
))

In [39]:
# 3rd call - final

response = gemini_client.models.generate_content(
    model='models/gemini-3-flash-preview',
    contents=messages,
    config=types.GenerateContentConfig(
        tools=[types.Tool(function_declarations=tools)],
        system_instruction=common.SINGLE_TOOL_SYSTEM_PROMPT,
    )
)

In [41]:
parts = response.candidates[0].content.parts
print(parts[0].text)

To create a dashboard in Evidently, you must first have an existing **Project** with at least one **Report** saved to it. Dashboards use these Reports as their data source to plot metrics over time or across experiments.

Every Project comes with a default dashboard that is initially empty. You populate it by adding **Panels** (visual widgets like line plots, bar charts, or counters).

### 1. Prerequisites
Before you can see data on a dashboard, you must:
1. Create a Workspace and a Project.
2. Run an evaluation (Report or Test Suite).
3. Save the results to the Project using the `project.add_report()` or `project.add_test_suite()` methods.

### 2. Methods to Create/Customize a Dashboard
You can add panels to your dashboard in two ways:

#### A. Using the Python API (Available in OSS, Cloud, and Enterprise)
You can define your dashboard layout as code. This involves adding panels to the project’s dashboard configuration.

**Example:**
```python
from evidently.report import Report
from 

In [43]:
message_history = [
    types.Content(
        parts=[types.Part(text=common.DEFAULT_QUESTION)],
        role="user"
    )
]

iteration_number = 1

while True:
    print(f'iteration number {iteration_number}...')

    response = gemini_client.models.generate_content(
        model='models/gemini-3-flash-preview',
        contents=message_history,
        config=types.GenerateContentConfig(
            tools=[types.Tool(function_declarations=tools)],
            system_instruction=common.SINGLE_TOOL_SYSTEM_PROMPT,
        )
    )

    parts = response.candidates[0].content.parts
    message_history.append(types.Content(role="model", parts=parts))
    
    tool_call_results = []

    for part in parts:
        if part.function_call is not None:
            function_call = part.function_call
            print(f'executing {function_call.name}({function_call.args})...')
            tool_call_output = make_call(function_call)
            tool_call_results.append(tool_call_output)

        if part.text is not None:
            text = part.text
            print('ASSISTANT:', text)

    if len(tool_call_results) > 0:
        message_history.append(types.Content(role="user", parts=tool_call_results))
    else:
        break

    iteration_number = iteration_number + 1
    print()

iteration number 1...
executing search({'query': 'how to create a dashboard'})...

iteration number 2...
executing get_file({'filename': 'docs/platform/dashboard_overview.mdx'})...

iteration number 3...
executing get_file({'filename': 'docs/platform/dashboard_add_panels.mdx'})...

iteration number 4...
executing get_file({'filename': 'docs/platform/dashboard_add_panels_ui.mdx'})...

iteration number 5...
ASSISTANT: To create a dashboard in Evidently, you first need to have a **Project** with at least one **Report** saved to it, as dashboards visualize data extracted from these Reports.

You can create and customize dashboards using either the Python API (as code) or the User Interface (no-code).

### 1. Prerequisite: Save Reports to a Project
Before adding panels to a dashboard, ensure you have initialized a workspace and saved a report:
```python
from library.workspace import Workspace
from library.report import Report
from library.metric_preset import DataDriftPreset

# 1. Create/co

In [50]:
from agent_gemini import GeminiAgent

In [51]:
agent = GeminiAgent(
    llm_client=gemini_client,
    model='models/gemini-3-flash-preview',
    instructions=common.SINGLE_TOOL_SYSTEM_PROMPT,
    tool_pairs=tool_pairs
)

In [ ]:
messages = agent.loop(common.DEFAULT_QUESTION)

iteration number 1...
executing search({'query': 'how to create a dashboard'})...

iteration number 2...
executing get_file({'filename': 'docs/platform/dashboard_overview.mdx'})...

iteration number 3...
executing search({'query': 'how to create a project and save report to it'})...

iteration number 4...
executing get_file({'filename': 'quickstart_ml.mdx'})...

iteration number 5...
